This notebook serves to explore various methods of detecting textures in images. The emphasis here is on finding representations that separate a texture from its background: The classifier used is included simply to provide a measure of representation quality.

Image spec.:
    - Do not include axes.
    - (15, 5) plots where necessary (title in Illustrator).
    - Grayscale single-channel images.

#### Introduction

For the purposes of this document, texture can be characterised as a distinct configuration of the following factors:
    - Intensity
    - Periodicity
    - Non-periodic variation (i.e. randomness)
    - Orientation
    - Direction
    - Size

Additional information about a texture may also be derived from:
    - Position relative to the camera.
    - Orientation relative to the camera.
    - The texture's background.
    - The transition between texture and background.
    
Before examining specific filtering methods, we will briefly list what methods are available for texture analysis, along with their motivation.

*Peak-counting*
    - Suppress local non-maximum intensities.
    - Apply a threshold.
    - Count the peaks in the vicinity.
    
*Autocorrelation*
    - Compute correlations between pixel intensities across some period at a given orientation (e.g. every other pixel, every 5 pixels etc.).
    
*Fourier analysis*
    - Map the image from the intensity domain to the spatial frequency domain
    
*Co-occurence matrices, with summary statistics*
    - Compute the relative frequency with which two distinct intensities reccur at a given orientation and radial distance.
    - Summary statistics - energy, entropy, inertia, correlation, local homogeneity.
    
*Texture energy via Laws' filters*
    - Begin with three kernels designed to extract edges, spots, and local intensity.
    - Take these kernels' outer products to derive a set of masks.
    - Convolve these masks with the images, then smooth (why smooth?).
    - Square the resulting smoothed values to obtain energy measures.
    - Combine the various energy measurements to produce features useful for classification.
    
*Texture energy via eigenfilters*
    - Compute the covariance matrix over a texture region.
    - Obtain the eigenvectors of this covariance matrix.
    - Use these eigenvectors as kernels with which to extract energy measures.
    
*Local rank correlation*

*Forced-choice method*

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

import numpy as np

from skimage.exposure import equalize_adapthist
from skimage.transform import resize

In [ ]:
# Display the image we'll be working with
from scipy.misc import imread

img_dir = "./Data/170812_RustAnalysis/"
sav_dir = "./Results/012_TextureAnalysis/"
raw_image = imread(img_dir + '001.png')[:, :, :-1]
raw_mask  = imread(img_dir + '006.png')
raw_mask  = np.logical_and(raw_mask[:,:,0] < 20, raw_mask[:, :, 1] > 240)
raw_mask  = (resize(raw_mask, raw_image.shape, mode = 'reflect') > 0.1)[:,:,0].astype(np.float32)

In [ ]:
plt.figure(figsize = (10, 6))
plt.imshow(raw_image)
plt.axis('off');

In [ ]:
plt.figure(figsize = (10, 6))
plt.imshow(raw_mask)
plt.axis('off');

### Equalize image

In [ ]:
equalized_image = equalize_adapthist(raw_image, 101)

plt.figure(figsize = (10, 6))
plt.imshow(equalized_image)
plt.axis('off');

In [ ]:
# Display a patch of the rust
rust_element = equalized_image[130:160, 260:-150]
plt.figure(figsize = (10, 6))
plt.imshow(rust_element)
plt.axis('off');

In this image I perceive multiple levels of variation. There are variations in colour with periods of between 2 and 40 pixels, which inclines me to think that a 40 x 40 region should be sufficient for our purposes. We could use another size, but this is fine for our purposes at present.

In [ ]:
rust_subelement = rust_element[10:30, 20:40]
plt.figure(figsize = (6, 6))
plt.imshow(rust_subelement)
plt.axis('off');

In [ ]:
import tensorflow as tf

class convolutional_softmax_classifier:
    
    def __init__(self, kernel_size, in_channels):
        
        self.graph = tf.Graph()

        with self.graph.as_default():
            with tf.name_scope('Inputs'):
                self.input_image = tf.placeholder(tf.float32, [None, None, None, in_channels]) # n-channel
                self.input_mask  = tf.placeholder(tf.float32, [None, None, None]) # Single-channel

            with tf.name_scope('Variables'):
                self.weights     = tf.Variable(tf.truncated_normal(shape  = [kernel_size, kernel_size, in_channels, 2],
                                                                  mean   = 0.,
                                                                  stddev = 1.))
                self.biases      = tf.Variable(tf.zeros(shape = [1, 1, 1, 2]))

            with tf.name_scope('Model'):
                with tf.name_scope('Standardization'):
                    channel_mean       = tf.expand_dims(tf.reduce_mean(self.input_image, axis = -1), axis = -1)
                    channel_var        = tf.reduce_mean(tf.subtract(self.input_image, channel_mean)**2)
                    standardized_input = (self.input_image - channel_mean)/tf.sqrt(channel_var)
                    standardized_mask  = self.input_mask/tf.reduce_max(self.input_mask)

                with tf.name_scope('LinearProjection'):
                    logits      = tf.nn.conv2d(standardized_input, self.weights,
                                               strides = [1, 1, 1, 1], padding = 'SAME') + self.biases
                    self.predictions = tf.nn.softmax(logits, dim = -1)

                with tf.name_scope('Loss'):
                    ohe_logits  = tf.transpose(tf.reshape(logits, shape = [2, -1]))
                    ohe_mask    = tf.reshape(standardized_mask, shape = [-1])
                    ohe_labels  = tf.stack([ohe_mask, 1 - ohe_mask], axis = -1)
                    self.loss   = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = ohe_labels,
                                                                                         logits = ohe_logits))

            with tf.name_scope('Optimization'):
                self.optimize    = tf.train.GradientDescentOptimizer(0.05).minimize(self.loss)

In [ ]:
ksize = 5
training_steps = 5001

rgb_c = convolutional_softmax_classifier(ksize, in_channels = 3)

with tf.Session(graph = rgb_c.graph) as session:
    session.run(tf.global_variables_initializer())
    fd        = {rgb_c.input_image : equalized_image[np.newaxis, :, :, :],
                 rgb_c.input_mask  : raw_mask[np.newaxis, :, :] }
    
    for step in range(training_steps):
        _, l = session.run([rgb_c.optimize, rgb_c.loss], feed_dict = fd)
        if step % 500 == 0:
            print('Loss at step {:^3d}: {:^5.2f}'.format(step, l))
            
    p_rgb = session.run(rgb_c.predictions, feed_dict = fd)

In [ ]:
plt.imshow(p_rgb[0, :, :, 1], cmap = 'gray')
plt.axis('off');

### Eigenfilters

Our objective now is to compute the covariance matrix of this image patch. A problem is that we are working with a 20x20x3 patch, for which the corresponding covariance matrix contains 1.6 million elements. Our purpose in deriving the covariance matrix is to compute its eigenvectors (i.e. the principal components of the rust texel in intensity space).

We could conceivably move to a different colour space to reduce the number of elements (e.g. grayscale or a channel of HSL or RGB), however in doing this we would lose information that may be valuable. 

Let's see how we can do with a 3x3 patch.

In [ ]:
patch_size = 7
patches    = [ rust_element[i:i+patch_size, j:j + patch_size, :].flatten()
               for i in range(rust_element.shape[0] - patch_size)
               for j in range(rust_element.shape[1] - patch_size) ]
patches    = np.stack(patches, axis = 0)

C = np.cov(patches, rowvar = False)
eigenvalues, principal_components = np.linalg.eig(C)

plt.figure(figsize = (15, 5))
plt.hist(eigenvalues, bins = 21); plt.xlabel('Component standard deviation'); plt.ylabel('Frequency');

We can see that one component points in a direction , with 18 components pointing in directions with little variation.

In [ ]:
from scipy.signal import convolve, gaussian

def apply_smoothing(image, ksize = 5):
    kernel_size = ksize
    n_std       = 3
    K           = np.tensordot(gaussian(kernel_size, std = kernel_size/n_std)[:, np.newaxis],
                               gaussian(kernel_size, std = kernel_size/n_std)[np.newaxis, :],
                               axes = [1, 0])
    return convolve(image, K, mode = 'same')

def apply_filters(image, filters):
    """
        Filters is a list of filter kernels to be convolved with an image.
        Returns a stack of image channels, with each channel corresponding to its respective filter.
    """
    return [convolve(image.astype(np.float32), f, mode = 'valid').squeeze() for f in filters]

n_filters  = 10
eigfilters = [principal_components[i].reshape([patch_size, patch_size, 3])
              for i in np.flip(eigenvalues.argsort(), axis = 0)[:n_filters]]

filtered_images = [np.pad(apply_smoothing(img)**2, [[patch_size//2, patch_size//2]]*2, mode = 'reflect')
                   for img in apply_filters(raw_image, eigfilters)]

In [ ]:
f, axs = plt.subplots(n_filters, 2, figsize = (15, 25))

for i, ax in enumerate(axs.flatten()):
    if i % 2 == 0:
        ax.imshow(raw_image)
        ax.axis('off');
    else:
        ax.imshow(filtered_images[i//2], cmap = 'gray')
        ax.axis('off');

In [ ]:
eigfilter_input = np.stack(filtered_images, axis = -1)
eig_c = convolutional_softmax_classifier(ksize, in_channels = eigfilter_input.shape[-1])

with tf.Session(graph = eig_c.graph) as session:
    session.run(tf.global_variables_initializer())
    fd        = {eig_c.input_image : eigfilter_input[np.newaxis, :, :, :],
                 eig_c.input_mask  : raw_mask[np.newaxis, :, :] }
    
    for step in range(training_steps):
        _, l = session.run([eig_c.optimize, eig_c.loss], feed_dict = fd)
        if step % 500 == 0:
            print('Loss at step {:^3d}: {:^5.2f}'.format(step, l))
            
    p_eig = session.run(eig_c.predictions, feed_dict = fd)

In [ ]:
plt.imshow(p_eig[0, patch_size//2:-patch_size//2, patch_size//2:-patch_size//2, 0], cmap = 'gray')
plt.axis('off');

### Laws' filters

In [ ]:
L5 = np.array([1, 4, 6, 4, 1])
E5 = np.array([-1, 2, 0, 2, 1])
S5 = np.array([1, 0, 2, 0, -1])
R5 = np.array([1, -4, 6, -4, 1])
W5 = np.array([-1, 2, 0, -2, 1])

Laws_filters = [np.tensordot(f1[:, np.newaxis], f2[np.newaxis, :], axes = [1, 0])
                for f1 in [L5, E5, S5, R5, W5]
                for f2 in [L5, E5, S5, R5, W5]]

In [ ]:
f, axs = plt.subplots(5, 5, figsize = (15, 15))

for i, ax in enumerate(axs.flatten()):
    ax.imshow(Laws_filters[i], cmap = 'gray')
    ax.axis('off');

In [ ]:
Laws_filters = [np.stack([f, f, f], axis = -1) for f in Laws_filters]

filtered_images = [np.pad(apply_smoothing(img)**2, [[patch_size//2, patch_size//2]]*2, mode = 'reflect')
                   for img in apply_filters(equalized_image, Laws_filters)]

In [ ]:
f, axs = plt.subplots(len(Laws_filters), 2, figsize = (15, 50))

for i, ax in enumerate(axs.flatten()):
    if i % 2 == 0:
        ax.imshow(equalized_image)
        ax.axis('off');
    else:
        ax.imshow(filtered_images[i//2], cmap = 'gray')
        ax.axis('off');

In [ ]:
plt.imshow(filtered_images[-2], cmap = 'gray')
plt.axis('off');

In [ ]:
plt.imshow(filtered_images[-2][1:-1, 1:-1]*-equalized_image[:,:,0], cmap = 'gray')
plt.axis('off');